In [1]:
"""
Authored by Darren Colby and used for testing plotting functions
"""

import altair as alt
import pandas as pd
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
df = pd.read_json("../../data/preprocessed_comments.json")
df.head()

,text,date,sentiment
0,thank god bless,2023-03-02 11:33:47+00:00,0.7506
1,bless,2023-03-01 20:35:35+00:00,0.4215
2,absolutely awesome sail way,2023-03-01 14:33:12+00:00,0.6590
3,extremely awesome people fur family touch warm...,2023-03-01 13:30:20+00:00,0.8883
4,kudo series tico parent extra step respective ...,2023-03-01 01:04:53+00:00,0.8957


In [4]:
def get_forecast(df: pd.DataFrame, lead: int):
    ts_df = df[["date", "x"]].rename(columns={"date": "ds", "x": "y"})
    ts_df["ds"] = ts_df.ds.dt.tz_localize(None)

    df2 = df.copy(deep=True)
    df2["forecast"] = "Actual"

    model = Prophet()
    model.fit(ts_df)
    future_df = model.make_future_dataframe(periods=lead)
    forecast = model.predict(future_df)[["ds", "yhat"]].rename(columns={"ds": "date", 
                                                                    "yhat": "x"})
    forecast["forecast"] = "Predicted"
    return pd.concat([forecast, df2[["date", "x", "forecast"]]])


def plot_ts(df: pd.DataFrame, title: str, y_col: str, y_title: str, caption: bool, 
            lead: int):
    df_to_plot = get_forecast(df, lead)

    # Have to convert to datetime and remove timezone for altair
    df_to_plot["date"] = pd.to_datetime(df_to_plot.date, utc=True)

    # The base plot
    ts_plot = alt.Chart(df_to_plot).mark_line().encode(
        alt.X(f"yearmonthdatehours(date):T", 
              title="",

              # Only displays the first and last ticks and labels
              # Found suggestion on Stack Overflow from user jakevdp
              # https://stackoverflow.com/questions/59699412/altair-display-all-axis-ticks-but-only-some-tick-labels
              axis=alt.Axis(tickCount=df_to_plot.shape[0])),
        alt.Y(y_col, 
              title=y_title),
        color=alt.Color(
            "forecast:N",
            legend=alt.Legend(title="")
        )
    ).properties(
        title=title
    )


    # Only adds caption for sentiment time series
    if caption:
        # The caption, which is not a property in the Chart class
        # Found solution on Stack Overflow by user jakevdp
        # https://stackoverflow.com/questions/57244390/how-to-add-a-subtitle-to-an-altair-generated-chart
        caption = alt.Chart(
            {"values": [{"text": 
                        "Sentiment ranges from -1 for most negative to +1 for most positive"}]}
        ).mark_text(align="left").encode(
            text="text:N"
        )

        final_plot = alt.vconcat(
            ts_plot,
            caption
        )

        return final_plot
    
    return ts_plot


In [5]:
def plot_sentiment_ts(df: pd.DataFrame, lead:int):
    df_copy = df.copy(deep=True)
    df_copy["x"] = df_copy.sentiment

    return plot_ts(df_copy, "Changes in Sentiment", "x", "Sentiment", True, lead)

plot_sentiment_ts(df, 3)

11:53:53 - cmdstanpy - INFO - Chain [1] start processing
11:53:53 - cmdstanpy - INFO - Chain [1] done processing


alt.VConcatChart(...)

In [7]:
def plot_comment_ts(df: pd.DataFrame, lead: int):
    new_df = df.copy(deep=True)
    new_df = new_df.date.value_counts().rename_axis("date").reset_index(name="x")

    return plot_ts(new_df, "Comments accross time", "x", "Comments", False, lead)

plot_comment_ts(df, 3)

alt.Chart(...)

In [8]:
def plot_comment_cumsum_ts(df: pd.DataFrame, lead: int):
    new_df = df.copy(deep=True)
    new_df = new_df.date.value_counts().rename_axis("date").reset_index(name="x")
    new_df["date"] = pd.to_datetime(new_df.date)
    new_df.sort_values("date", inplace=True)
    new_df["x"] = new_df["x"].cumsum()

    return plot_ts(new_df, "Comments accross time", "x", "Comments", False, lead)

plot_comment_cumsum_ts(df, 3)

11:57:32 - cmdstanpy - INFO - Chain [1] start processing
11:57:32 - cmdstanpy - INFO - Chain [1] done processing


alt.Chart(...)

In [9]:
pd.read_json("../../data/similarity_data.json")

,w55xmZLWfBg,NsscBcwjTNg,DEtyL4lXp7s,ECHlvUyaXFU,4znhKBm5oOA
0,"[thank god bless, 2023-03-02T11:33:47Z]","[super angel love, 2023-03-02T03:58:51Z]","[definition groomed run, 2023-03-02T18:16:14Z]",[dodgy dessert look fabulous ingenious way bri...,"[miss shadow, 2023-03-02T12:54:58Z]"
1,"[bless, 2023-03-01T20:35:35Z]","[thank earth angel, 2023-03-02T01:08:30Z]","[look tell outlaw greyhound race clearly good,...","[beautiful thank, 2023-03-02T04:45:03Z]","[think pup run like cast bone break, 2023-03-0..."
2,"[absolutely awesome sail way, 2023-03-01T14:33...","[Buggy, 2023-03-02T01:03:57Z]",[thank rescue paul nice joy appreciation thing...,"[dummy, 2023-03-02T03:43:30Z]","[love tony, 2023-03-01T17:54:30Z]"
3,[extremely awesome people fur family touch war...,"[thank yougod bless, 2023-03-02T00:26:35Z]",[course hard time dog paul wear clothe probabl...,[god bless family incredible rescue dog live a...,[beautiful sweet happy dog watch run free true...
4,[kudo series tico parent extra step respective...,"[heart heaven, 2023-03-02T00:07:16Z]","[beautifulthank patience paulo glad happy, 202...","[dog gorgeous baking excellent, 2023-03-01T08:...","[happy find love tony, 2023-03-01T13:29:54Z]"
5,"[tico, 2023-02-28T15:01:39Z]",[amazing wonderful guy difference dog live bea...,"[paul find way look happyawoooooo paul, 2023-0...","[bless kindness, 2023-03-01T00:51:38Z]",[think wonderful people foster animal impossib...
6,"[cute puppy, 2023-02-28T13:12:58Z]","[thank, 2023-03-01T17:09:00Z]","[heart need morning, 2023-03-01T12:31:17Z]","[crew love mama fantastic treat, 2023-02-28T20...","[tony, 2023-02-28T14:33:52Z]"
7,[love communitybuilde nature dog dog dog bring...,"[great human love, 2023-03-01T12:36:51Z]","[guy wonderful inspiring, 2023-03-01T10:29:13Z]","[adorable, 2023-02-28T19:42:55Z]","[love shelter tube blessing, 2023-02-28T02:22:..."
8,"[brilliant play date lovely thing, 2023-02-26T...",[amazing young man god bless thank save precio...,"[god bless guy amazing money ill, 2023-03-01T0...",[compassion joyful way love help amazing story...,"[right know right, 2023-02-28T02:20:05Z]"
9,"[dog right playdate, 2023-02-26T16:34:32Z]",[thank love devotion dedication sacrifice god ...,"[think nervous embarrassed wear outfit dog, 20...","[extravagant lol smile, 2023-02-28T16:37:06Z]","[thank love watch cookie, 2023-02-27T23:43:32Z]"
